# Preprocessing + 30-day Readmission Notebook

This focused notebook extracts and documents the preprocessing steps used to build the 30-day readmission label and the pooled static + time-series features. It is a smaller workspace for iterative experiments (safe to run on a dev machine).

Key contents:
- DATA_DIR setup and quick preview of CSVs
- 30-day readmission label computation
- Loading patients/diagnoses/chartevents/labevents (with date parsing)
- Age computation using `anchor_age` and `anchor_year`
- Static features and pooled time-series feature construction
- Imputation, scaling, and simple CV training for baseline models

In [1]:
# DATA_DIR and quick preview (copy of template)
import os, pandas as pd
DATA_DIR = '/Users/yuchenzhou/documents/duke/compsci526/final_proj/mimic-iv-3.1'
print('DATA_DIR =', DATA_DIR)
paths = {
    'admissions': os.path.join(DATA_DIR, 'hosp', 'admissions.csv'),
    'patients': os.path.join(DATA_DIR, 'hosp', 'patients.csv'),
    'diagnoses_icd': os.path.join(DATA_DIR, 'hosp', 'diagnoses_icd.csv'),
    'chartevents': os.path.join(DATA_DIR, 'icu', 'chartevents.csv'),
    'labevents': os.path.join(DATA_DIR, 'hosp', 'labevents.csv'),
}
from IPython.display import display
for name, p in paths.items():
    print('---', name, '->', p)
    if os.path.exists(p):
        try:
            df = pd.read_csv(p, nrows=5)
            print('columns:', list(df.columns))
            display(df.head(3))
        except Exception as e:
            print('读取失败:', e)
    else:
        print('NOT FOUND')

DATA_DIR = /Users/yuchenzhou/documents/duke/compsci526/final_proj/mimic-iv-3.1
--- admissions -> /Users/yuchenzhou/documents/duke/compsci526/final_proj/mimic-iv-3.1/hosp/admissions.csv
columns: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag']


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P19UTS,EMERGENCY ROOM,HOSPICE,Medicaid,English,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0


--- patients -> /Users/yuchenzhou/documents/duke/compsci526/final_proj/mimic-iv-3.1/hosp/patients.csv
columns: ['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000058,F,33,2168,2020 - 2022,NaN


--- diagnoses_icd -> /Users/yuchenzhou/documents/duke/compsci526/final_proj/mimic-iv-3.1/hosp/diagnoses_icd.csv
columns: ['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version']


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9


--- chartevents -> /Users/yuchenzhou/documents/duke/compsci526/final_proj/mimic-iv-3.1/icu/chartevents.csv
columns: ['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime', 'storetime', 'itemid', 'value', 'valuenum', 'valueuom', 'warning']


,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,18704,2180-07-23 12:36:00,2180-07-23 14:45:00,226512,39.4,39.4,kg,0
1,10000032,29079034,39553978,18704,2180-07-23 12:36:00,2180-07-23 14:45:00,226707,60,60.0,Inch,0
2,10000032,29079034,39553978,18704,2180-07-23 12:36:00,2180-07-23 14:45:00,226730,152,152.0,cm,0


--- labevents -> /Users/yuchenzhou/documents/duke/compsci526/final_proj/mimic-iv-3.1/hosp/labevents.csv
columns: ['labevent_id', 'subject_id', 'hadm_id', 'specimen_id', 'itemid', 'order_provider_id', 'charttime', 'storetime', 'value', 'valuenum', 'valueuom', 'ref_range_lower', 'ref_range_upper', 'flag', 'priority', 'comments']


,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,2704548,50931,P69FQC,2180-03-23 11:51:00,2180-03-23 15:56:00,___,95.0,mg/dL,70.0,100.0,NaN,ROUTINE,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
1,2,10000032,NaN,36092842,51071,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN
2,3,10000032,NaN,36092842,51074,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN


In [2]:
# 30-day readmission label (copy of working code)
import pandas as pd
def _parse_dates_if_exists(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    return df
admissions = pd.read_csv(os.path.join(DATA_DIR, 'hosp', 'admissions.csv'))
admissions = _parse_dates_if_exists(admissions, ['admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime'])
admissions = admissions.sort_values(['subject_id', 'admittime']).reset_index(drop=True)
readmit_30d = []
for idx, row in admissions.iterrows():
    subject_id = row['subject_id']
    hadm_id = row['hadm_id']
    dischtime = row['dischtime']
    if pd.isna(dischtime):
        readmit_30d.append(0)
        continue
    future_admissions = admissions[
        (admissions['subject_id'] == subject_id) &
        (admissions['hadm_id'] != hadm_id) &
        (admissions['admittime'] > dischtime)
    ]
    if not future_admissions.empty:
        next_admit = future_admissions.iloc[0]['admittime']
        days_to_readmit = (next_admit - dischtime).total_seconds() / (24*3600)
        readmit_30d.append(1 if days_to_readmit <= 30 else 0)
    else:
        readmit_30d.append(0)
admissions['readmit_30d'] = readmit_30d
print('Computed readmit_30d, distribution:')
print(admissions['readmit_30d'].value_counts(normalize=True).mul(100).round(2))

Computed readmit_30d, distribution:
readmit_30d
0    80.3
1    19.7
Name: proportion, dtype: float64


In [ ]:
# Load patients/diagnoses and time-series (with date parsing)
def _read(path, **kwargs):
    if os.path.exists(path):
        return pd.read_csv(path, **kwargs)
    raise FileNotFoundError(path)
patients = _read(os.path.join(DATA_DIR, 'hosp', 'patients.csv'))
diagnoses = _read(os.path.join(DATA_DIR, 'hosp', 'diagnoses_icd.csv'))
print('Reading small heads:')
print(patients.head(2))
print(diagnoses.head(2))

In [ ]:
# Static features and pooled time-series features (copy of functions)
admissions = admissions.merge(patients[['subject_id','anchor_age','anchor_year']], on='subject_id', how='left')
admissions['admit_year'] = admissions['admittime'].dt.year
admissions['age'] = admissions['anchor_age'] + (admissions['admit_year'] - admissions['anchor_year'])
admissions['age'] = admissions['age'].clip(lower=0)
admissions['los_days'] = (admissions['dischtime'] - admissions['admittime']).dt.total_seconds()/(3600*24)
static_df = admissions[['subject_id','hadm_id','age','los_days','readmit_30d']].drop_duplicates().reset_index(drop=True)
print('static_df sample:')
print(static_df.head())
# For full time-series pooling we reference chartevents and labevents; those are large and may be filtered in practice

## Preprocessing summary
- Parsed `admittime`/`dischtime`/`charttime` where present.
- Constructed `readmit_30d` by checking the next admission within 30 days after discharge for the same `subject_id`.
- Computed approximate `age` using `anchor_age` and `anchor_year`.
- Built `static_df` with `age`, `los_days`, and the `readmit_30d` label.
- Time-series pooling and imputation code (copied to template) produces `X_ts_imputed` and pooled summary features.

Detailed reasoning for each preprocessing step (why I do it):

- Date parsing (admittime/dischtime/charttime):
  - Why: time calculations (length-of-stay, event windows, time-based labels) require proper datetime types. Parsing with `errors='coerce'` avoids crashes when a few malformed rows exist.
  - Risk/caveat: coerce converts bad values to NaT; inspect parsing rates for unexpected drops.

- Compute 30-day readmission label (`readmit_30d`):
  - Why: the project's primary outcome is whether the patient returns within 30 days. Using the same `subject_id` and comparing `admittime` > `dischtime` is robust to overlapping stays and ordering.
  - Choices made: we consider the *next* admission chronologically for that patient; alternative definitions (any admission within 30 days, excluding planned readmissions) can be implemented later.
  - Caveat: planned readmissions and transfers should be excluded for some analyses; add filters if you have those flags.

- Age estimation using `anchor_age` / `anchor_year`:
  - Why: this MIMIC-IV export doesn't include `dob`, so `anchor_age` + (admit_year - anchor_year) gives an approximate age at admission.
  - Caveat: this is approximate and may be off by a year or two due to anchor-year bucketing.

- Static feature selection (age, gender, LOS):
  - Why: these are baseline covariates known to correlate with readmission risk and are cheap to compute. Keep the set small for the smoke test; expand later.

- Time-window selection and pooling (72-hour window, 1H resample):
  - Why: using a fixed early-window (e.g., first 72 hours) standardizes inputs and focuses on early inpatient signals that could predict readmission risk. Hourly resampling balances temporal resolution and computational cost.
  - Alternatives: longer windows or different pooling (min/max/last/percentiles) depending on the problem framing.

- Imputation strategy (linear interpolation then mean fill):
  - Why: clinical time series are irregular; interpolation fills short gaps while forward/backward filling and mean imputation handle longer missing stretches. This is a pragmatic choice for baseline models.
  - Caveat: imputation can leak future info if misused (we only interpolate inside the admission window from observed times). For causal/time-to-event work, prefer models that handle irregular time or explicit missingness indicators.

- Pooling into summary stats (mean, std, min, max):
  - Why: reduces time-series to compact fixed-length vectors suitable for classical ML baselines (LogReg, RF) and quick experiments. These pooled features are interpretable and fast.
  - Next step: use sequence models (LSTM/Transformer) on full time-series for richer temporal modeling.

- Class weighting and stratified CV:
  - Why: readmission is imbalanced (~20% positive in your preview). Using `class_weight='balanced'` and stratified splits makes training and evaluation more robust and comparable.

Caveats & next improvements:
- Add explicit cohort exclusions (planned readmissions, transfers, hospice, death during stay) depending on your study design.
- Save parsing logs (rows parsed, NaT count) to detect data drift or broken exports.
- Consider more advanced imputation (KNN, learned imputers) or models that explicitly handle missingness.
- For reproducibility, pin the random seed and record data slices used for smoke tests.

In [ ]:
# --------- Time-series pooling, imputation, and small smoke test (nrows) ---------
print('Starting smoke test: time-series pooling + imputation + quick CV...')
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd

# Default seed if not set earlier
if 'RANDOM_SEED' not in globals():
    RANDOM_SEED = 42

# Ensure core dataframes exist; if not, load/compute them (makes this cell runnable standalone)
if 'static_df' not in globals():
    print('static_df not found in memory — constructing from admissions & patients (this may take a few seconds)')
    def _parse_dates_if_exists(df, cols):
        for col in cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
        return df
    def _read(path, **kwargs):
        if os.path.exists(path):
            return pd.read_csv(path, **kwargs)
        raise FileNotFoundError(path)

    admissions = pd.read_csv(os.path.join(DATA_DIR, 'hosp', 'admissions.csv'))
    admissions = _parse_dates_if_exists(admissions, ['admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime'])
    admissions = admissions.sort_values(['subject_id', 'admittime']).reset_index(drop=True)

    # compute readmit_30d if missing
    if 'readmit_30d' not in admissions.columns:
        readmit_30d = []
        for idx, row in admissions.iterrows():
            subject_id = row['subject_id']
            hadm_id = row['hadm_id']
            dischtime = row['dischtime']
            if pd.isna(dischtime):
                readmit_30d.append(0)
                continue
            future_admissions = admissions[
                (admissions['subject_id'] == subject_id) &
                (admissions['hadm_id'] != hadm_id) &
                (admissions['admittime'] > dischtime)
            ]
            if not future_admissions.empty:
                next_admit = future_admissions.iloc[0]['admittime']
                days_to_readmit = (next_admit - dischtime).total_seconds() / (24*3600)
                readmit_30d.append(1 if days_to_readmit <= 30 else 0)
            else:
                readmit_30d.append(0)
        admissions['readmit_30d'] = readmit_30d
    
    # load patients and merge anchor_age/year
    patients = _read(os.path.join(DATA_DIR, 'hosp', 'patients.csv'))
    if 'anchor_age' in patients.columns and 'anchor_year' in patients.columns:
        admissions = admissions.merge(patients[['subject_id','anchor_age','anchor_year']], on='subject_id', how='left')
        admissions['admit_year'] = admissions['admittime'].dt.year
        admissions['age'] = admissions['anchor_age'] + (admissions['admit_year'] - admissions['anchor_year'])
        admissions['age'] = admissions['age'].clip(lower=0)
    else:
        # Fallback if anchor_age/year missing
        admissions['age'] = np.nan

    admissions['los_days'] = (admissions['dischtime'] - admissions['admittime']).dt.total_seconds()/(3600*24)
    static_df = admissions[['subject_id','hadm_id','age','los_days','readmit_30d']].drop_duplicates().reset_index(drop=True)
    print('Constructed static_df; sample:')
    print(static_df.head())

# Read small chunks of chartevents / labevents for smoke testing (adjust nrows as needed)
ce_path = os.path.join(DATA_DIR, 'icu', 'chartevents.csv')
le_path = os.path.join(DATA_DIR, 'hosp', 'labevents.csv')

# Guard: if files absent, skip
if not os.path.exists(ce_path) or not os.path.exists(le_path):
    print('chartevents or labevents not found; skipping smoke test of time-series.')
else:
    print('Loading small slices of chartevents/labevents (this may still take a few seconds)...')
    try:
        ce = pd.read_csv(ce_path, usecols=['subject_id','hadm_id','charttime','itemid','valuenum','value'], nrows=200000, low_memory=False)
    except Exception:
        ce = pd.read_csv(ce_path, nrows=200000, low_memory=False)
    try:
        le = pd.read_csv(le_path, usecols=['labevent_id','subject_id','hadm_id','charttime','itemid','valuenum','value'], nrows=50000, low_memory=False)
    except Exception:
        le = pd.read_csv(le_path, nrows=50000, low_memory=False)

    # use lowercase 'h' for resampling compatibility warnings
    ce = _parse_dates_if_exists(ce, ['charttime'])
    le = _parse_dates_if_exists(le, ['charttime'])

    # Define simple item lists (same as template) — these are examples and can be customized
    vital_items_of_interest = {
        'heart_rate': [211, 220045],
        'sys_bp': [220179, 51],
        'dias_bp': [220180, 8368],
        'resp_rate': [220210, 618],
        'temp': [223761, 678],
        'spo2': [220277]
    }
    lab_items_of_interest = {
        'creatinine': [50912],
        'glucose': [807, 823],
        'wbc': [730],
    }

    def get_patient_events_smoke(hadm_id, window_hours=72, resample_freq='1h'):
        # Use the small loaded slices (ce, le) — return None if no data
        adm_row = admissions[admissions['hadm_id']==hadm_id]
        if adm_row.empty:
            return None
        t0 = adm_row.iloc[0]['admittime']
        t_end = t0 + pd.Timedelta(hours=window_hours)
        ce_sub = ce[(ce['hadm_id']==hadm_id) & (ce['charttime']>=t0) & (ce['charttime']<=t_end)] if not ce.empty else pd.DataFrame()
        le_sub = le[(le['hadm_id']==hadm_id) & (le['charttime']>=t0) & (le['charttime']<=t_end)] if not le.empty else pd.DataFrame()
        rows = []
        for var, itemids in vital_items_of_interest.items():
            if ce_sub.empty:
                continue
            tmp = ce_sub[ce_sub['itemid'].isin(itemids)][['charttime','valuenum']].dropna()
            if tmp.empty:
                continue
            tmp = tmp.rename(columns={'charttime':'time','valuenum':var})
            tmp[var] = pd.to_numeric(tmp[var], errors='coerce')
            rows.append(tmp.set_index('time')[var])
        for var, itemids in lab_items_of_interest.items():
            if le_sub.empty:
                continue
            tmp = le_sub[le_sub['itemid'].isin(itemids)][['charttime','valuenum']].dropna()
            if tmp.empty:
                continue
            tmp = tmp.rename(columns={'charttime':'time','valuenum':var})
            tmp[var] = pd.to_numeric(tmp[var], errors='coerce')
            rows.append(tmp.set_index('time')[var])
        if not rows:
            return None
        combined = pd.concat(rows, axis=1)
        combined = combined.resample(resample_freq).mean()
        combined = combined[:t0 + pd.Timedelta(hours=window_hours)]
        return combined

    # Build a small dataset for first N hadm_ids
    hadm_ids_small = static_df['hadm_id'].dropna().unique()[:50]
    X_static_small = []
    X_ts_small = []
    y_small = []
    all_cols = list(vital_items_of_interest.keys()) + list(lab_items_of_interest.keys())

    for hid in hadm_ids_small:
        s = static_df[static_df['hadm_id']==hid]
        if s.empty:
            continue
        ts = get_patient_events_smoke(hid, window_hours=72, resample_freq='1h')
        if ts is None:
            continue
        for c in all_cols:
            if c not in ts.columns:
                ts[c] = np.nan
        ts = ts[all_cols]
        # ensure TIMESTEPS
        if len(ts) < 72:
            pad_len = 72 - len(ts)
            if len(ts) == 0:
                continue
            pad_df = pd.DataFrame(np.nan, index=pd.date_range(ts.index[-1]+pd.Timedelta(hours=1), periods=pad_len, freq='1h'), columns=ts.columns)
            ts = pd.concat([ts, pad_df])
        else:
            ts = ts.iloc[:72]
        # Static selection: age and los_days (if los_days NaN, fill 0)
        s_age = s['age'].iloc[0] if not pd.isna(s['age'].iloc[0]) else 0.0
        s_los = s['los_days'].iloc[0] if not pd.isna(s['los_days'].iloc[0]) else 0.0
        X_static_small.append(np.array([s_age, s_los], dtype=float))
        X_ts_small.append(ts.values.astype(float))
        y_small.append(int(s['readmit_30d'].iloc[0]))

    X_static_small = np.array(X_static_small)
    X_ts_small = np.array(X_ts_small)
    y_small = np.array(y_small)

    if len(y_small) == 0:
        print('No time-series data available for the small sample with the chosen item lists. Consider increasing nrows or adjusting itemid lists.')
    else:
        print('Built small dataset shapes:', X_static_small.shape, X_ts_small.shape, y_small.shape)

        # Impute time-series
        def impute_time_series_array(X_ts):
            N, T, F = X_ts.shape
            X_imputed = X_ts.copy()
            for i in range(N):
                df_ts = pd.DataFrame(X_ts[i], columns=[f'f{j}' for j in range(F)])
                df_ts = df_ts.interpolate(method='linear', limit_direction='both', axis=0).ffill().bfill()
                df_ts = df_ts.fillna(df_ts.mean())
                X_imputed[i] = df_ts.values
            return X_imputed

        X_ts_imputed_small = impute_time_series_array(X_ts_small)

        # Pool time-series features
        def pool_time_series_features(X_ts):
            N, T, F = X_ts.shape
            feats = []
            for i in range(N):
                arr = X_ts[i]
                mean = np.nanmean(arr, axis=0)
                std = np.nanstd(arr, axis=0)
                mn = np.nanmin(arr, axis=0)
                mx = np.nanmax(arr, axis=0)
                # Handle all-NaN columns by replacing with zeros (safe for small baseline models)
                mean = np.nan_to_num(mean, nan=0.0)
                std = np.nan_to_num(std, nan=0.0)
                mn = np.nan_to_num(mn, nan=0.0)
                mx = np.nan_to_num(mx, nan=0.0)
                feats.append(np.concatenate([mean, std, mn, mx]))
            return np.array(feats)

        X_pool_small = pool_time_series_features(X_ts_imputed_small)

        # Scale static features
        scaler_small = StandardScaler()
        X_static_scaled_small = scaler_small.fit_transform(X_static_small)

        X_final_small = np.hstack([X_static_scaled_small, X_pool_small])
        print('Final pooled feature shape:', X_final_small.shape)

        # Quick train/test split and evaluation
        try:
            if len(np.unique(y_small))>1:
                Xtr, Xte, ytr, yte = train_test_split(X_final_small, y_small, test_size=0.3, random_state=RANDOM_SEED, stratify=y_small)
            else:
                Xtr, Xte, ytr, yte = train_test_split(X_final_small, y_small, test_size=0.3, random_state=RANDOM_SEED)
        except Exception:
            Xtr, Xte, ytr, yte = train_test_split(X_final_small, y_small, test_size=0.3, random_state=RANDOM_SEED)

        lr = LogisticRegression(max_iter=1000, class_weight='balanced')
        rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, class_weight='balanced')

        lr.fit(Xtr, ytr)
        rf.fit(Xtr, ytr)

        lr_p = lr.predict_proba(Xte)[:,1]
        rf_p = rf.predict_proba(Xte)[:,1]

        print('Smoke test results:')
        if len(set(yte))>1:
            print('LR AUROC:', roc_auc_score(yte, lr_p), 'AUPRC:', average_precision_score(yte, lr_p))
            print('RF AUROC:', roc_auc_score(yte, rf_p), 'AUPRC:', average_precision_score(yte, rf_p))
        else:
            print('Insufficient label variation in test split to compute AUROC/AUPRC.')

print('Smoke test complete.')


Starting smoke test: time-series pooling + imputation + quick CV...
static_df not found in memory — constructing from admissions & patients (this may take a few seconds)
Constructed static_df; sample:
   subject_id   hadm_id  age  los_days  readmit_30d
0    10000032  22595853   52  0.786111            0
1    10000032  22841357   52  1.015278            1
2    10000032  29079034   52  2.222222            1
3    10000032  25742920   52  1.754167            0
4    10000068  25022803   19  0.298611            0
Loading small slices of chartevents/labevents (this may still take a few seconds)...


/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:134: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  combined = combined.resample(resample_freq).mean()
/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:161: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pad_df = pd.DataFrame(np.nan, index=pd.date_range(ts.index[-1]+pd.Timedelta(hours=1), periods=pad_len, freq='1H'), columns=ts.columns)
/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:134: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  combined = combined.resample(resample_freq).mean()
/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:161: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pad_df = pd.DataFrame(np.na

Built small dataset shapes: (34, 2) (34, 72, 9) (34,)
Final pooled feature shape: (34, 38)


/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:134: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  combined = combined.resample(resample_freq).mean()
/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:161: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pad_df = pd.DataFrame(np.nan, index=pd.date_range(ts.index[-1]+pd.Timedelta(hours=1), periods=pad_len, freq='1H'), columns=ts.columns)
/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:134: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  combined = combined.resample(resample_freq).mean()
/var/folders/v0/1lrkgf3s4d3_x4q14m3pmy840000gn/T/ipykernel_92993/2906703046.py:161: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pad_df = pd.DataFrame(np.na

NameError: name 'RANDOM_SEED' is not defined